# Data reading

In [93]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-11 10:24:41--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-06-11 10:24:42 (26.7 MB/s) - ‘input.txt.4’ saved [1115394/1115394]



In [94]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [95]:
print(len(text), len(text.split()), len(set(text)))

1115394 202651 65


# Data processing

In [96]:
chars = sorted(set(text))
vocab_size = len(chars)
print(f"{vocab_size=}\n<S>{'|'.join(chars)}<E>")

vocab_size=65
<S>
| |!|$|&|'|,|-|.|3|:|;|?|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z<E>


In [97]:
# creating vocabulary and mapping and reverse mapping
itos = {i:c for i, c in enumerate(chars)}
stoi = {c:i for i, c in itos.items()}
encode = lambda s : [stoi[c] for c in s]
decode = lambda e : ''.join([itos[i] for i in e])

sample_text = "what's up"
print(encode(sample_text))
print(decode(encode(sample_text)))

[61, 46, 39, 58, 5, 57, 1, 59, 54]
what's up


## few notes about tokenizer usage

Try other encodings as well.
e.g.
* SentencePiece
* Also BPE's fast implementation is tiktoken used in GPT models created by OpenAI
* OpenAI's tokenizer demo - https://platform.openai.com/tokenizer
* Third party tokenizer demo - https://tiktokenizer.vercel.app/
* tiktoken repo - https://github.com/openai/tiktoken
* sample tiktoken usage cookbook - https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb


---


**ToDo** - Learn about BPE tokenizer process in depth.

In [98]:
#!pip install tiktoken
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
print(enc.encode(sample_text))
print(enc.decode(enc.encode(sample_text)))


50257
[10919, 338, 510]
what's up


## data processing continued (with character encoding)

In [99]:
import torch
#encode text into token
data = torch.tensor(encode(text), dtype=torch.long)
print(f"{data.shape=}, {data.dtype=}")
print(data[:100])

data.shape=torch.Size([1115394]), data.dtype=torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [100]:
# train, val split
split = int(0.9*len(data))
train_data = data[:split]
val_data = data[split:]
print(f"{train_data.shape=}, {val_data.shape=}")

train_data.shape=torch.Size([1003854]), val_data.shape=torch.Size([111540])


In [101]:
# get batches of data in block_size

block_size = 8 # also called as context length
batch_size = 4 # for parallel processing

torch.manual_seed(1337)
def get_batch(split):
  data = train_data if split=='train' else val_data
  batch_ixs = torch.randint(0, len(data) - block_size, (batch_size,))
  x = torch.stack([data[ix : ix + block_size] for ix in batch_ixs])
  y = torch.stack([data[ix + 1: ix + 1 + block_size] for ix in batch_ixs])
  '''
  if x indices are  [3,4,5,6,7,8,9, 10]
  y indices will be [4,5,6,7,8,9,10,11]

  torch.stack will just stack in one extra dimension in front
  '''
  return x, y

xb, yb = get_batch('train')
print(f"{xb.shape=}, {yb.shape=}")
print(xb)
print(yb)

for bt in range(batch_size):
  for bl in range(block_size):
    context = xb[bt, :bl+1]
    output  = yb[bt, bl]
    print(f"{decode(context.numpy())} --> {decode([output.item()])}")


xb.shape=torch.Size([4, 8]), yb.shape=torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
L --> e
Le --> t
Let --> '
Let' --> s
Let's -->  
Let's  --> h
Let's h --> e
Let's he --> a
f --> o
fo --> r
for -->  
for  --> t
for t --> h
for th --> a
for tha --> t
for that -->  
n --> t
nt -->  
nt  --> t
nt t --> h
nt th --> a
nt tha --> t
nt that -->  
nt that  --> h
M --> E
ME --> O
MEO --> :
MEO: --> 

MEO:
 --> I
MEO:
I -->  
MEO:
I  --> p
MEO:
I p --> a


# lets start with simple bigram model

learn more about torch libraries - specifically start with nn.Module

In [102]:
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [103]:
import torch.nn as nn
import torch.nn.functional as F
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding = nn.Embedding(vocab_size, vocab_size)

  def forward(self, x, targets):
    logits = self.token_embedding(x) # converts shape(B, T) to shape(B, T, C) | B-batch, T-?, C-Channel(Embedding_size)
    B,T,C = logits.shape
    # read documentation of cross_entropy for reason of this reshaping -
    # https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html#torch.nn.functional.cross_entropy
    logits = logits.view(B*T, C)
    targets = targets.view(B*T) # or just -1
    loss = F.cross_entropy(logits, targets)

    return logits, loss

model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(f"{logits=}, {loss=}")

logits=tensor([[ 1.6347, -0.0518,  0.4996,  ...,  0.2432,  1.1519,  0.9950],
        [ 0.3418, -0.9276,  1.2381,  ...,  1.5018, -0.5266,  0.2354],
        [ 0.1479, -0.4333,  0.5203,  ...,  0.3302,  1.5454,  1.3778],
        ...,
        [-0.5693, -0.0735,  0.7743,  ..., -0.0815, -1.1445, -0.0623],
        [ 0.4658, -0.2573, -1.0673,  ...,  1.2439,  1.3471,  1.6910],
        [-0.4553,  0.0139,  0.9309,  ...,  0.0290, -0.7568,  0.8701]],
       grad_fn=<ViewBackward0>), loss=tensor(5.0364, grad_fn=<NllLossBackward0>)
